In [15]:
%reload_ext autoreload
#%load_ext autoreload
%autoreload 2

# Question 3

Write a simple (supervised) deep classifier to train and test using the dataset collected in Q1.
How will you divide your dataset into training and test sets.
What data-augmentation techniques will to use for out-of-distribution (unseen) images?
c. Please test accuracy on the attached, rd_test_dataset zipped face images, and save the output to a .csv file.
https://drive.google.com/file/d/1jcdByJPkAGq9JsgsdLqeyLwI4Yl6plOf/view? usp=sharing
d. Explain your accuracy scores, and add analysis based on what you proposal in Q2.

# Answer
1. We divide data into training, validate and test set using the ratios of 70%,20%,10% respectively
2. We read image vectors through ImageDataGenerator that has built methods for generating batches of tensor image data with real-time data augmentation.
3. The trained model had lower validation accuracy than on training data. The test data performance using the 70,20,10% split showed to be worse than either validation or training but closer to validation accuracy.
4. Applied to the test files given on google drive. Downloaded the dataset, made a filelist, form image tensors from the image files. Ran model.predict, saved the results as a dataframe in a csv
5. To get a decent model, I would obviously need to run this on larger data.
6. I also tried another approach to improve the model by cropping the image regions containing only the face and building the model from the cropped images. Not sure which over is better on the test dataset given but they seem to perform similarly on their validation and test splits.
7. Due to the small amount of training data, the model seem to have low and test validation accuracy. Need to run on larger collections to see any difference.

# Method 1
Using the whole image for classification of fake/real. Sometimes the background can also help with the detection

In [16]:
from keras.models import Sequential 
from keras.layers import Conv2D,MaxPool2D,Flatten,Dense,Dropout
from keras.preprocessing.image import ImageDataGenerator
from keras.utils import to_categorical
import os
import numpy as np
from PIL import Image
import pandas as pd

In [17]:
# A basic CNN model used for the discrimination
def create_cnnmodel():
    model = Sequential()

    model.add(Conv2D(32,(3,3),input_shape = (200,200,3),activation = 'relu',padding='same',))
    model.add(MaxPool2D(pool_size=(2,2)))
    model.add(Dropout(0.2))

    model.add(Conv2D(64, (3, 3), activation='relu', padding='same'))
    model.add(MaxPool2D((2, 2)))
    model.add(Dropout(0.2))

    model.add(Conv2D(128, (3, 3), activation='relu', padding='same'))
    model.add(MaxPool2D((2, 2)))
    model.add(Dropout(0.2))

    model.add(Flatten())

    model.add(Dense(128, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(1, activation='sigmoid'))

    model.compile(optimizer = 'adam',loss = 'binary_crossentropy',metrics = ['accuracy'])
    return model

In [18]:

import os
import splitfolders
import shutil

def split_data(indir,outdir,modeldir):
    if not os.path.exists(outdir):
        os.makedirs(outdir)
    else:
        #delete all the models, splits, etc. to create fresh splits
        shutil.rmtree(outdir)
        os.makedirs(outdir)
        
    if not os.path.exists(modeldir):
        os.makedirs(modeldir)
    else:
        shutil.rmtree(modeldir)
        os.makedirs(modeldir)
    splitfolders.ratio(indir,output=outdir,seed=40,ratio=(0.7,0.2,0.1), group_prefix=None)
    
    
def evaluate_performance(dirname,outresult,model):
    filelist=[]
    for root, dirs, files in os.walk(dirname, topdown=False):
            for name in files:
                 if (not name.startswith(".")):
                        path=root+"/"+name
                        filelist.append(path)
    #print(len(filelist),filelist)
    x = np.array([np.array((Image.open(fname)).resize((200,200))) for fname in filelist])
    resultarray=model.predict(x)
    dict={"Sample":filelist,"FakeorReal?":resultarray[:,0]}
    df=pd.DataFrame(dict)
    print(df.head())
    df.to_csv(outresult)  
    


In [19]:
#training the model using the training and validation splits
def fit_model(model,trainpath,validatepath,testpath):
    datagen = ImageDataGenerator(rescale=1.0/255.0)

    train = datagen.flow_from_directory(trainpath,
                                    class_mode='binary',
                                    batch_size=64,
                                    target_size=(200,200))

    validate = datagen.flow_from_directory(validatepath,
                                    class_mode='binary',
                                    batch_size=64,
                                    target_size=(200,200))

    history = model.fit_generator(train,
                              validation_data=(validate),
                              epochs = 50,
                              steps_per_epoch=len(train),
                              validation_steps=len(validate))
    test = datagen.flow_from_directory(testpath,
                                    class_mode='binary',
                                    batch_size=64,
                                    target_size=(200,200))
    scoreSeg=model.evaluate_generator(test, 4,verbose=1)
    print(scoreSeg)
    return scoreSeg


    

In [24]:
#all paths
indir="../data/sampled"
splitdir="../data/sampledsplits"
modeldir="../models"
trainpath=splitdir+"/train/"
validatepath=splitdir+"/val/"
testpath=splitdir+"/test/"
modelpath=modeldir+"/model.h5"
print(modelpath)
testdir='../data/rd_test_dataset'
outresult="../results/result.csv"



../models/model.h5


In [8]:

#build, train and evaluate model on sample data
#This code splits the data folder into train, validation and test splits in the 0.7,0.2,0.1 ratios
#Evaluating the performance on the given dataset on https://drive.google.com/file/d/1jcdByJPkAGq9JsgsdLqeyLwI4Yl6plOf/view? 
#this directory is copied in data/rd_test_dataset
split_data(indir,splitdir,modeldir)
model=create_cnnmodel()
fit_model(model,trainpath,validatepath,testpath)
model.save(modelpath)
evaluate_performance(testdir,outresult,model)

2022-06-30 21:39:31.548156: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
/var/folders/6s/75106hxs7bddl69mnq1djf640000gn/T/ipykernel_62737/187624938.py:15: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  history = model.fit_generator(train,


Found 70 images belonging to 2 classes.
Found 20 images belonging to 2 classes.
Epoch 1/50
2/2 [==============================] - 4s 771ms/step - loss: 0.8417 - accuracy: 0.5714 - val_loss: 2.6348 - val_accuracy: 0.5000
Epoch 2/50
2/2 [==============================] - 3s 521ms/step - loss: 5.3707 - accuracy: 0.5000 - val_loss: 0.7544 - val_accuracy: 0.5000
Epoch 3/50
2/2 [==============================] - 3s 3s/step - loss: 1.5909 - accuracy: 0.5000 - val_loss: 0.6962 - val_accuracy: 0.5000
Epoch 4/50
2/2 [==============================] - 3s 3s/step - loss: 0.7461 - accuracy: 0.5000 - val_loss: 0.6922 - val_accuracy: 0.5000
Epoch 5/50
2/2 [==============================] - 3s 3s/step - loss: 0.6843 - accuracy: 0.6286 - val_loss: 0.6925 - val_accuracy: 0.5000
Epoch 6/50
2/2 [==============================] - 3s 3s/step - loss: 0.6882 - accuracy: 0.5714 - val_loss: 0.6922 - val_accuracy: 0.5000
Epoch 7/50
2/2 [==============================] - 3s 3s/step - loss: 0.6912 - accuracy: 0.54

/var/folders/6s/75106hxs7bddl69mnq1djf640000gn/T/ipykernel_62737/187624938.py:24: UserWarning: `Model.evaluate_generator` is deprecated and will be removed in a future version. Please use `Model.evaluate`, which supports generators.
  scoreSeg=model.evaluate_generator(test, 4,verbose=1)


4/4 [==============================] - 1s 223ms/step - loss: 0.7337 - accuracy: 0.7000
[0.7336750030517578, 0.699999988079071]
1/1 [==============================] - 0s 330ms/step
                            Sample  FakeorReal?
0  ../data/rd_test_dataset/016.jpg          1.0
1  ../data/rd_test_dataset/002.jpg          1.0
2  ../data/rd_test_dataset/003.jpg          1.0
3  ../data/rd_test_dataset/017.jpg          1.0
4  ../data/rd_test_dataset/001.jpg          1.0


In [9]:
#testing to see the model can be read back and reused
import tensorflow as tf
from tensorflow import keras
load_model=tf.keras.models.load_model(modelpath)

# Method 2

Trying on only face regions as detected by a face detector

In [20]:
#crops the region containing face in all the images of a directory and copies the cropped images out to another.

from matplotlib import pyplot
from matplotlib.patches import Rectangle
from mtcnn.mtcnn import MTCNN
from PIL import Image
from tensorflow.keras.preprocessing.image import array_to_img


def crop_faces(dirname,outdir,detector):
    if not os.path.exists(outdir):
            os.makedirs(outdir)
    for root, dirs, files in os.walk(dirname, topdown=False):
        for name in files:
            if (not name.startswith(".")):
                path=root+"/"+name
                outpath=outdir+"/"+name
                #print(path)
                pixels = pyplot.imread(path)
                img= array_to_img(pixels)
                faces = detector.detect_faces(pixels)
                draw_image_with_boxes(path, faces)
                imageBox = img.getbbox()
                #imageBox = Image.fromarray(pixels).getbbox()
                imwidth=imageBox[3]
                imheight=imageBox[2]
                if (len(faces)>=1):
                    coordinates = tuple(faces[0]['box']) 
                    x, y, width, height = coordinates
                    x=max(x,0)
                    y=max(y,0)
                    xmax=min(x+width,imwidth)
                    ymax=min(y+height,imheight)
                    coordinates=[x,y,xmax,ymax]
                    #print(coordinates,pixels.shape,imageBox) 
                    try:
                        Image.fromarray(pixels).crop(coordinates).save(outpath)
                    except (IndexError or SystemError):
                            print('Face detection error image ' ,path,subs,files)
                else:
                   # print(coordinates,pixels.shape,imageBox) 
                    try:
                        img.save(outpath)
                    except (IndexError or SystemError):
                            print('Face save error image ' ,path,subs,files)

def draw_image_with_boxes(filename, result_list):
    # load the image
    data = pyplot.imread(filename)
    # plot the image
   # print(type(data),data.shape)
    pyplot.imshow(data)
    # get the context for drawing boxes
    ax = pyplot.gca()
    # plot each box
    for result in result_list:
        # get coordinates
        x, y, width, height = result['box']
        # create the shape
        rect = Rectangle((x, y), width, height, fill=False, color='red')
        # draw the box
        ax.add_patch(rect)
    # show the plot
    pyplot.show()

def save_cropped_faces(dirname,outdir,detector):
    if detector is None:
        detector = MTCNN()
    if not os.path.exists(outdir):
        os.makedirs(outdir)
    else:
        #delete all the models, splits, etc. to create fresh splits
        shutil.rmtree(outdir)
        os.makedirs(outdir)
    crop_faces(dirname,outdir,detector)
    return detector
def create_cropped_dataset(topdir,outdir):
    realdir=topdir+"/real"
    fakedir=topdir+"/fake"
    realoutdir=outdir+"/real"
    fakeoutdir=outdir+"/fake"
    detector=save_cropped_faces(realdir,realoutdir,None)
    save_cropped_faces(fakedir,fakeoutdir,detector)
    return detector


In [23]:
sampledir="../data/sampled"
cropdir="../data/cropped"
cropsplitdir="../data/croppedsplits"
modeldir="../models"
trainpath=cropsplitdir+"/train/"
validatepath=cropsplitdir+"/val/"
testpath=cropsplitdir+"/test/"
croppedmodelpath=modeldir+"/modelcropped.h5"
print(croppedmodelpath)
testdir='../data/rd_test_dataset'
testcropdir="../data/croppedtest"
outcropresult="../results/resultcropped.csv"

../models/modelcropped.h5


In [21]:
#Generate face-specific dataset called cropped
#Does a split of the cropped image data
#create a new instance of the CNN Model to test with face regions only
#train the cropped region-based CNN model
detector=create_cropped_dataset(sampledir,cropdir)
split_data(cropdir,cropsplitdir,modeldir)
croppedmodel=create_cnnmodel()
fit_model(croppedmodel,trainpath,validatepath,testpath)
croppedmodel.save(croppedmodelpath)
save_cropped_faces(testdir,testcropdir,detector)
evaluate_performance(testcropdir,outcropresult,croppedmodel)

1/1 [==============================] - 0s 273ms/step
                        Sample   FakeorReal?
0  ../data/croppedtest/016.jpg  1.000000e+00
1  ../data/croppedtest/002.jpg  1.000000e+00
2  ../data/croppedtest/003.jpg  3.011414e-22
3  ../data/croppedtest/017.jpg  1.000000e+00
4  ../data/croppedtest/001.jpg  1.000000e+00
